--- Day 2: Password Philosophy ---
Your flight departs in a few days from the coastal airport; the easiest way down to the coast from here is via toboggan.

The shopkeeper at the North Pole Toboggan Rental Shop is having a bad day. "Something's wrong with our computers; we can't log in!" You ask if you can take a look.

Their password database seems to be a little corrupted: some of the passwords wouldn't have been allowed by the Official Toboggan Corporate Policy that was in effect when they were chosen.

To try to debug the problem, they have created a list (your puzzle input) of passwords (according to the corrupted database) and the corporate policy when that password was set.

For example, suppose you have the following list:

1-3 a: abcde
1-3 b: cdefg
2-9 c: ccccccccc
Each line gives the password policy and then the password. The password policy indicates the lowest and highest number of times a given letter must appear for the password to be valid. For example, 1-3 a means that the password must contain a at least 1 time and at most 3 times.

In the above example, 2 passwords are valid. The middle password, cdefg, is not; it contains no instances of b, but needs at least 1. The first and third passwords are valid: they contain one a or nine c, both within the limits of their respective policies.

How many passwords are valid according to their policies?

In [1]:
#!fsharp
#r "nuget: FParsec"
open FParsec

let input = System.IO.File.ReadAllLines "./input.txt"

In [1]:
#!fsharp
type ValidationRule = { MinCount : int; MaxCount : int; Letter : char }

type Password = string

let validationRule minCount maxCount letter = { MinCount = minCount ; MaxCount = maxCount ; Letter = letter}

let ruleParser : Parser<ValidationRule, unit> = 
    pipe3 
        (pint32) 
        ((pchar '-') >>. pint32) 
        (spaces >>. asciiLower) 
        validationRule

let passwordParser : Parser<Password, unit> = restOfLine false

let inputParser : Parser<(ValidationRule * Password), unit> =
    pipe2
        (ruleParser .>> pchar ':' .>> spaces)
        passwordParser
        (fun a b -> (a, b))

let passwordIsValid (rule: ValidationRule) (password: Password) : bool =
    String.filter (fun c -> c = rule.Letter) password
    |> String.length
    |> fun x -> rule.MinCount <= x && x <= rule.MaxCount

let ruleAndPassword = 
    List.ofArray input 
    |> List.map (run inputParser)
    |> List.map (
        function
        | Success (r, _, _) -> r
        | _ -> failwith "Error parsing input" 
        )

List.where (fun (rule, password) -> passwordIsValid rule password ) ruleAndPassword |> List.length

638

--- Part Two ---
While it appears you validated the passwords correctly, they don't seem to be what the Official Toboggan Corporate Authentication System is expecting.

The shopkeeper suddenly realizes that he just accidentally explained the password policy rules from his old job at the sled rental place down the street! The Official Toboggan Corporate Policy actually works a little differently.

Each policy actually describes two positions in the password, where 1 means the first character, 2 means the second character, and so on. (Be careful; Toboggan Corporate Policies have no concept of "index zero"!) Exactly one of these positions must contain the given letter. Other occurrences of the letter are irrelevant for the purposes of policy enforcement.

Given the same example list from above:

1-3 a: abcde is valid: position 1 contains a and position 3 does not.
1-3 b: cdefg is invalid: neither position 1 nor position 3 contains b.
2-9 c: ccccccccc is invalid: both position 2 and position 9 contain c.
How many passwords are valid according to the new interpretation of the policies?

In [1]:
#!fsharp
let passwordIsValidActually (rule: ValidationRule) (password: Password) : bool =
    (Seq.tryItem (rule.MinCount - 1) password, Seq.tryItem (rule.MaxCount - 1) password )
    |> fun (a, b) -> 
    match ( Option.map ((=) rule.Letter) a, Option.map ((=) rule.Letter) b) with
    | (Option.Some true, Option.Some false) -> true
    | (Option.Some false, Option.Some true) -> true
    | _ -> false

List.where (fun (rule, password) -> passwordIsValidActually rule password ) ruleAndPassword |> List.length

699